# Experiments on KuHar Resampled to 30Hz

This notebook will perform crossvalidation experiments using the KuHar dataset at 30 Hz as training dataset. It will contain the following steps:

1. Quick load train, test and validation CSV subsets from the balanced KuHar dataset at 30 Hz using `PandasDatasetsIO` helper
2. Quick load train, test and validation CSV subsets from other relevant datasets using `PandasDatasetsIO` helper
3. Subclassing the `Dataset` interface using `PandasMultiModalDataset`
4. Apply the fourier transform on all the datasets
5. Train SVM, KNN and Random Forest classification models on the KuHar dataset in both time and frequency domains
6. Evaluate SVM, KNN and Random Forest classification models on other dataset in both time and frequency domains

The experiments will evaluate the performance of SVM, KNN and RF models trained on a balanced KuHar dataset and tested on other datasets in both time and frequency domains.

## Common imports and definitions

In [1]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

2022-09-05 13:16:17.127130: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 13:16:17.127151: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data from base dataset
Change the path here to use in other dataset as base

In [2]:
# Path for KuHar resampled to 30Hz view with the same activities (and labels numbers)
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
dataset_path = Path("../data/views/KuHar/resampled_view_30Hz")

Once paths is defined, we can load the CSV as pandas dataframes

In [3]:
# Kuhar dataframes
train_base, validation_base, test_base = PandasDatasetsIO(dataset_path).load()

Letś take a look in the train dataframes

In [4]:
train_base.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-89,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,0.003620,-0.023688,-0.002657,0.010371,-0.005858,0.010423,-0.019593,0.000303,0.045433,...,0.003377,23.235,23.223,26.260,26.249,0,300,1,2100,1051
1,1,-0.005823,0.012494,-0.012503,-0.002116,0.025957,-0.012833,-0.025845,-0.011941,0.012807,...,0.003056,56.292,56.292,59.245,59.245,0,300,1,5700,1037
2,2,-0.039278,0.003864,0.008927,-0.024887,0.022435,0.003431,-0.038931,0.003359,0.009394,...,0.003442,27.268,27.267,30.290,30.291,0,300,1,2700,1075
3,3,-0.001728,-0.018312,0.013927,0.015426,0.007332,-0.012372,0.006893,-0.002433,0.012821,...,-0.001294,39.421,39.420,42.441,42.440,0,300,6,3900,1008
4,4,-0.022981,0.014871,-0.036310,0.033512,-0.016733,0.019930,-0.016637,0.007568,-0.002753,...,0.004560,23.703,23.703,26.656,26.656,0,300,1,2400,1038


## Creating a Librep dataset from pandas dataframes

Change the features to use in other datasets

In [5]:
# Kuhar features to select
features = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z"
]

# Creating the train dataset

combined_train_dset = PandasMultiModalDataset(
    pd.concat([train_base, validation_base, test_base], ignore_index=True),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

## Inspect sample

In [6]:
# Lets print the first sample of combined_train dataset.
# Is a tuple, with an vector of 540 elements as first element and the label as second
x = combined_train_dset[0]
print(x)

(array([ 3.62008887e-03, -2.36884079e-02, -2.65723909e-03,  1.03708716e-02,
       -5.85826359e-03,  1.04230261e-02, -1.95931668e-02,  3.02941467e-04,
        4.54334895e-02, -4.61346122e-03, -4.32958544e-02, -2.11811327e-03,
       -3.24331746e-02, -9.64189479e-04,  2.38423378e-02,  1.46571666e-02,
       -6.33010786e-02, -3.09536555e-02, -6.09642095e-03, -2.23327124e-02,
        4.73811398e-02,  8.10745709e-03, -1.46291624e-03, -3.52547565e-02,
       -3.65486640e-02,  3.89222511e-02, -5.64525903e-02,  1.71065679e-02,
       -1.88765233e-02,  1.56273664e-02,  6.68170176e-02,  1.86759574e-02,
       -4.15165877e-02,  1.55675209e-02,  4.57480490e-03, -1.56470432e-02,
        2.86984361e-02, -3.88362938e-03, -1.85153493e-02, -9.91992429e-03,
        1.05702633e-02, -5.34487311e-03,  2.19452625e-02,  2.31337120e-02,
       -7.65234924e-03, -5.43440699e-02, -2.68535392e-02,  1.13130045e-02,
        2.85217945e-02,  1.84220024e-02,  1.95164920e-02,  4.88774164e-03,
        6.26222956e-03, 

In [7]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [ 3.62008887e-03 -2.36884079e-02 -2.65723909e-03  1.03708716e-02
 -5.85826359e-03  1.04230261e-02 -1.95931668e-02  3.02941467e-04
  4.54334895e-02 -4.61346122e-03 -4.32958544e-02 -2.11811327e-03
 -3.24331746e-02 -9.64189479e-04  2.38423378e-02  1.46571666e-02
 -6.33010786e-02 -3.09536555e-02 -6.09642095e-03 -2.23327124e-02
  4.73811398e-02  8.10745709e-03 -1.46291624e-03 -3.52547565e-02
 -3.65486640e-02  3.89222511e-02 -5.64525903e-02  1.71065679e-02
 -1.88765233e-02  1.56273664e-02  6.68170176e-02  1.86759574e-02
 -4.15165877e-02  1.55675209e-02  4.57480490e-03 -1.56470432e-02
  2.86984361e-02 -3.88362938e-03 -1.85153493e-02 -9.91992429e-03
  1.05702633e-02 -5.34487311e-03  2.19452625e-02  2.31337120e-02
 -7.65234924e-03 -5.43440699e-02 -2.68535392e-02  1.13130045e-02
  2.85217945e-02  1.84220024e-02  1.95164920e-02  4.88774164e-03
  6.26222956e-03  1.74586067e-02  4.62809500e-03 -4.00069874e-02
  1.28236155e-03  1.63634494e-02 -7.79265175e-04  3.31124115e-02
 -6.2073271

## Fourier Transform

In [8]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [9]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")

### Use FFT in the base dataset

In [10]:
combined_train_dset_fft = transformer(combined_train_dset)

In [11]:
combined_train_dset[:][0]

array([[ 3.62008887e-03, -2.36884079e-02, -2.65723909e-03, ...,
         2.38507494e-03,  4.01598763e-03,  3.37713756e-03],
       [-5.82278332e-03,  1.24938221e-02, -1.25032413e-02, ...,
        -3.11192249e-03,  2.45944768e-03,  3.05583499e-03],
       [-3.92784081e-02,  3.86433489e-03,  8.92735084e-03, ...,
         4.96789877e-03,  3.30600047e-03,  3.44192435e-03],
       ...,
       [ 1.63371509e+00,  2.66028584e+00,  3.44735791e+00, ...,
         1.18921088e-01,  1.87933683e-02, -3.68293330e-01],
       [ 8.18879998e-01, -9.45890066e-01, -3.65675276e+00, ...,
         6.52041693e-01,  5.93416057e-01,  6.12118067e-01],
       [ 4.27490898e+00,  1.63163418e+00, -1.18631011e+00, ...,
        -7.60297782e-01, -6.13890826e-01, -6.25430027e-01]])

In [12]:
combined_train_dset_fft[:][0]

array([[3.13384994e-02, 1.68121633e-01, 9.05549650e-02, ...,
        5.92594004e-03, 1.10474894e-02, 7.03162882e-03],
       [2.30704315e-02, 1.23651598e-01, 6.28149230e-02, ...,
        8.26119084e-03, 3.20484735e-03, 5.24385686e-03],
       [7.81909007e-02, 7.24224293e-02, 1.34035957e-01, ...,
        1.17237633e-02, 8.03545237e-03, 2.37891069e-02],
       ...,
       [5.95110302e+00, 1.24723971e+01, 1.54138493e+01, ...,
        1.29725094e+00, 8.58205011e-01, 8.19612955e-01],
       [1.01743943e+01, 1.83103529e+00, 4.83797285e+00, ...,
        7.89111699e-01, 3.50632565e-01, 6.66859047e-01],
       [4.03706454e+01, 1.31043361e+01, 5.64244076e+01, ...,
        1.18233210e+00, 8.62392810e-01, 9.36655163e-01]])

## Loading data from a target dataset
In this case we use the MotionSense dataset resampled at 30 Hz

In [13]:
dataset_path = Path("../data/views/MotionSense/resampled_view_30Hz")

In [14]:
# Kuhar dataframes
train_target, validation_target, test_target = PandasDatasetsIO(dataset_path).load()

In [15]:
train_base.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-89,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,0.003620,-0.023688,-0.002657,0.010371,-0.005858,0.010423,-0.019593,0.000303,0.045433,...,0.003377,23.235,23.223,26.260,26.249,0,300,1,2100,1051
1,1,-0.005823,0.012494,-0.012503,-0.002116,0.025957,-0.012833,-0.025845,-0.011941,0.012807,...,0.003056,56.292,56.292,59.245,59.245,0,300,1,5700,1037
2,2,-0.039278,0.003864,0.008927,-0.024887,0.022435,0.003431,-0.038931,0.003359,0.009394,...,0.003442,27.268,27.267,30.290,30.291,0,300,1,2700,1075
3,3,-0.001728,-0.018312,0.013927,0.015426,0.007332,-0.012372,0.006893,-0.002433,0.012821,...,-0.001294,39.421,39.420,42.441,42.440,0,300,6,3900,1008
4,4,-0.022981,0.014871,-0.036310,0.033512,-0.016733,0.019930,-0.016637,0.007568,-0.002753,...,0.004560,23.703,23.703,26.656,26.656,0,300,1,2400,1038


In [16]:
# MotionSense features to select
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

# Creating the train dataset

combined_target_dset = PandasMultiModalDataset(
    pd.concat([train_target, validation_target, test_target], ignore_index=True),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [17]:
# Lets print the first sample of combined_target dataset.
# Is a tuple, with an vector of 180 elements as first element and the label as second
x = combined_target_dset[0]
print(x)

(array([-1.86833389e-01, -1.79195359e-01, -2.26435131e-01, -2.34763436e-01,
       -2.67823557e-01, -2.34534448e-01, -2.35421098e-01, -1.33759301e-01,
       -2.97124572e-01, -3.19298325e-01,  8.95068153e-03,  4.13667107e-02,
       -4.45930376e-01, -2.28045572e-01,  5.05584862e-02,  1.88233135e-01,
       -3.05589990e-02, -7.44910767e-02,  3.59969568e-02, -5.25122747e-02,
       -1.20435540e-01, -2.99962061e-01, -4.50567186e-01,  1.46823014e-01,
        1.64267921e-02,  1.00866929e-01,  1.56137079e-01, -9.38140351e-02,
       -2.79031325e-01, -2.71325809e-01, -1.99438544e-01, -3.22355437e-01,
       -2.81747706e-01, -1.88338075e-01, -1.75930166e-01, -2.58367241e-01,
       -4.13807163e-01, -6.38969102e-01, -6.66223601e-02,  6.10918606e-01,
        1.23116570e-01,  7.50942481e-02,  1.32656873e-01,  1.36065249e-01,
       -7.60303034e-02, -1.99809749e-01,  8.69800753e-03,  3.54754458e-01,
       -2.36973734e-01, -3.67774663e-01, -2.45674023e-01,  1.50712010e-01,
        1.44587441e-01, 

In [18]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [-1.86833389e-01 -1.79195359e-01 -2.26435131e-01 -2.34763436e-01
 -2.67823557e-01 -2.34534448e-01 -2.35421098e-01 -1.33759301e-01
 -2.97124572e-01 -3.19298325e-01  8.95068153e-03  4.13667107e-02
 -4.45930376e-01 -2.28045572e-01  5.05584862e-02  1.88233135e-01
 -3.05589990e-02 -7.44910767e-02  3.59969568e-02 -5.25122747e-02
 -1.20435540e-01 -2.99962061e-01 -4.50567186e-01  1.46823014e-01
  1.64267921e-02  1.00866929e-01  1.56137079e-01 -9.38140351e-02
 -2.79031325e-01 -2.71325809e-01 -1.99438544e-01 -3.22355437e-01
 -2.81747706e-01 -1.88338075e-01 -1.75930166e-01 -2.58367241e-01
 -4.13807163e-01 -6.38969102e-01 -6.66223601e-02  6.10918606e-01
  1.23116570e-01  7.50942481e-02  1.32656873e-01  1.36065249e-01
 -7.60303034e-02 -1.99809749e-01  8.69800753e-03  3.54754458e-01
 -2.36973734e-01 -3.67774663e-01 -2.45674023e-01  1.50712010e-01
  1.44587441e-01 -1.08431514e-01 -1.82670965e-02 -8.24199145e-02
 -3.54580921e-02 -2.73086366e-01 -3.19687428e-01 -1.72529165e-01
  2.8114138

In [19]:
combined_target_dset_fft = transformer(combined_target_dset)

In [20]:
combined_target_dset[:][0]

array([[-0.18683339, -0.17919536, -0.22643513, ...,  0.89880418,
         1.13925318,  0.27555584],
       [-0.05444212,  0.26009924,  0.0229331 , ...,  0.25302617,
         0.85886433,  0.79907488],
       [-0.00769597, -0.00951478,  0.05128415, ..., -0.04252899,
        -0.16110965,  0.12915699],
       ...,
       [-0.31903905, -0.01283569, -0.03048265, ..., -0.4407308 ,
        -0.45499816, -0.90586541],
       [ 0.34836533,  0.52526862,  0.05997598, ..., -0.29225946,
        -0.48266957,  0.63102237],
       [ 0.09118243,  0.2065321 ,  0.38206896, ...,  0.3682671 ,
         0.32344069,  0.98400094]])

In [21]:
combined_target_dset_fft[:][0]

array([[7.8524508 , 1.97250947, 0.93442469, ..., 1.89050909, 0.83709156,
        1.20154337],
       [0.6293334 , 2.21672832, 1.13328689, ..., 1.16967075, 3.53071988,
        1.06557213],
       [3.1927866 , 1.55762082, 0.60235897, ..., 0.55560622, 0.71680657,
        0.39816055],
       ...,
       [1.2537858 , 1.29129674, 0.93891406, ..., 1.1729674 , 0.78012212,
        1.21739774],
       [3.1272222 , 2.91728302, 2.09904721, ..., 1.26141987, 5.8859495 ,
        1.62984581],
       [1.5251454 , 1.44493422, 1.36297853, ..., 1.90046437, 1.3680962 ,
        1.49000302]])

## Train and evaluate Random Forest classifier

In [22]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, estimator_creation_kwags ={'n_estimators':100} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [23]:
result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383787.110437
    result:
    -   accuracy: 0.045436507936507935
        f1 score (macro): 0.016818995027824156
        f1 score (micro): 0.045436507936507935
        f1 score (weighted): 0.04321919662751411
    run id: 1
    start: 1662383780.8642237
    time taken: 6.24621319770813
-   end: 1662383793.3233356
    result:
    -   accuracy: 0.04325396825396825
        f1 score (macro): 0.015761902182489677
        f1 score (micro): 0.04325396825396825
        f1 score (weighted): 0.04184921365754909
    run id: 2
    start: 1662383787.1104388
    time taken: 6.212896823883057
-   end: 1662383799.5207946
    result:
    -   accuracy: 0.04801587301587302
        f1 score (macro): 0.0174952776748573
        f1 score (micro): 0.04801587301587302
        f1 score (weighted): 0.04646179261965033
    run id: 3
    start: 1662383793.3233373
    time taken: 6.197457313537598



In [24]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383803.1330159
    result:
    -   accuracy: 0.018849206349206348
        f1 score (macro): 0.00838629943502825
        f1 score (micro): 0.018849206349206348
        f1 score (weighted): 0.015334947538337371
    run id: 1
    start: 1662383799.52607
    time taken: 3.606945753097534
-   end: 1662383806.7514472
    result:
    -   accuracy: 0.014484126984126985
        f1 score (macro): 0.006214352600664
        f1 score (micro): 0.014484126984126985
        f1 score (weighted): 0.01136092159970597
    run id: 2
    start: 1662383803.1330173
    time taken: 3.6184298992156982
-   end: 1662383810.3817186
    result:
    -   accuracy: 0.014682539682539682
        f1 score (macro): 0.005975210949170334
        f1 score (micro): 0.014682539682539682
        f1 score (weighted): 0.012435315009096753
    run id: 3
    start: 1662383806.7514489
    time taken: 3.630269765853882



## Train and evaluate Support Vector Machine classifier

In [25]:
#from librep.estimators import SVC
from sklearn.svm import SVC

experiment = SimpleTrainEvalWorkflow(estimator=SVC, estimator_creation_kwags ={'C':3.0, 'kernel':"rbf"} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383818.7436101
    result:
    -   accuracy: 0.06746031746031746
        f1 score (macro): 0.026169140700448706
        f1 score (micro): 0.06746031746031746
        f1 score (weighted): 0.06513625971943839
    run id: 1
    start: 1662383810.387501
    time taken: 8.356109142303467
-   end: 1662383827.0515416
    result:
    -   accuracy: 0.06746031746031746
        f1 score (macro): 0.026169140700448706
        f1 score (micro): 0.06746031746031746
        f1 score (weighted): 0.06513625971943839
    run id: 2
    start: 1662383818.7436118
    time taken: 8.307929754257202
-   end: 1662383835.3636444
    result:
    -   accuracy: 0.06746031746031746
        f1 score (macro): 0.026169140700448706
        f1 score (micro): 0.06746031746031746
        f1 score (weighted): 0.06513625971943839
    run id: 3
    start: 1662383827.0515432
    time taken: 8.312101125717163



In [26]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383838.6799273
    result:
    -   accuracy: 0.0003968253968253968
        f1 score (macro): 0.00037700282752120643
        f1 score (micro): 0.0003968253968253968
        f1 score (weighted): 0.00016531274778211632
    run id: 1
    start: 1662383835.3715367
    time taken: 3.3083906173706055
-   end: 1662383841.9793289
    result:
    -   accuracy: 0.0003968253968253968
        f1 score (macro): 0.00037700282752120643
        f1 score (micro): 0.0003968253968253968
        f1 score (weighted): 0.00016531274778211632
    run id: 2
    start: 1662383838.6799288
    time taken: 3.2994000911712646
-   end: 1662383845.2919164
    result:
    -   accuracy: 0.0003968253968253968
        f1 score (macro): 0.00037700282752120643
        f1 score (micro): 0.0003968253968253968
        f1 score (weighted): 0.00016531274778211632
    run id: 3
    start: 1662383841.9793303
    time taken: 3.3125860691070557



## Train and evaluate K Neighbors Classifier classifier

In [27]:
#from librep.estimators import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

experiment = SimpleTrainEvalWorkflow(estimator=KNeighborsClassifier, estimator_creation_kwags ={'n_neighbors' :1} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383845.6730227
    result:
    -   accuracy: 0.02619047619047619
        f1 score (macro): 0.020005610318975332
        f1 score (micro): 0.02619047619047619
        f1 score (weighted): 0.0190382685159935
    run id: 1
    start: 1662383845.2975938
    time taken: 0.3754289150238037
-   end: 1662383845.881996
    result:
    -   accuracy: 0.02619047619047619
        f1 score (macro): 0.020005610318975332
        f1 score (micro): 0.02619047619047619
        f1 score (weighted): 0.0190382685159935
    run id: 2
    start: 1662383845.6730263
    time taken: 0.2089695930480957
-   end: 1662383846.0922508
    result:
    -   accuracy: 0.02619047619047619
        f1 score (macro): 0.020005610318975332
        f1 score (micro): 0.02619047619047619
        f1 score (weighted): 0.0190382685159935
    run id: 3
    start: 1662383845.8819976
    time taken: 0.21025323867797852



In [28]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662383846.2455273
    result:
    -   accuracy: 0.006349206349206349
        f1 score (macro): 0.0038318764219853903
        f1 score (micro): 0.006349206349206349
        f1 score (weighted): 0.003757367713780119
    run id: 1
    start: 1662383846.0987105
    time taken: 0.14681673049926758
-   end: 1662383846.3874962
    result:
    -   accuracy: 0.006349206349206349
        f1 score (macro): 0.0038318764219853903
        f1 score (micro): 0.006349206349206349
        f1 score (weighted): 0.003757367713780119
    run id: 2
    start: 1662383846.245529
    time taken: 0.1419672966003418
-   end: 1662383846.5284188
    result:
    -   accuracy: 0.006349206349206349
        f1 score (macro): 0.0038318764219853903
        f1 score (micro): 0.006349206349206349
        f1 score (weighted): 0.003757367713780119
    run id: 3
    start: 1662383846.387498
    time taken: 0.14092087745666504

